# What 

This is related to #269.

To do recommendation exrtaction I need to complete a few tasks:

- [x] Web scraping of TSB for its recommendations
- [x] Safety action section extraction for ATSB and TSB (depending on if TSB recommendation dataset is in fact complete)
- [x] Safety action section parsing to gedt the recommendations
- [ ] Integration into the rest of the pipeline

In [ ]:
import importlib
import os
import shutil

import engine.gather.WebsiteScraping as WebsiteScraping
import engine.extract.ReportExtracting as ReportExtracting

import pandas as pd
from tqdm import tqdm

tqdm.pandas()
importlib.reload(WebsiteScraping)

# Web scraping of TSB

Firstly I need to scrape the recommendations of the TSB website.

This will be done in the `WebsiteScraping` module.

I am not sure if this recommendations dataset is complete or not.

In [ ]:
importlib.reload(WebsiteScraping)

recommendation_scraper = WebsiteScraping.TSBRecommendationsScraper('tsb_recommendations', refresh=False)

recommendation_scraper.extract_recommendations_from_website()

In [ ]:
tsb_recommendations = pd.read_pickle('tsb_recommendations')
tsb_recommendations

How does the current recommendations looks like

In [ ]:
taic_recommendations =  pd.read_pickle('../../output/taic_recommendations.pkl')

example_taic_recommendation = taic_recommendations.loc[213, 'recommendations']
example_taic_recommendation


After looking through the recommendations. I think that they are a compelte amount. Even if there are some larger gaps.

I have spot checked the rail and the gaps there seem to check out as the none of the reports in that few years had any recommendations.

# Safety Action section extraction

As found above we have a complete recommendation dataset for TSB. So we only need to scrape the ATSB reports for recommendations.recommendation_scraper

This can be done by reading the safety actions section I believe.

Itm ight even be possible to do it with just regex.

In [ ]:
parsed_reports

In [ ]:
report_titles = pd.read_pickle('../../output/report_titles.pkl')
parsed_reports = pd.read_pickle('../../output/parsed_reports.pkl')
all_reports = report_titles.merge(parsed_reports, how='right', on='report_id')
atsb_reports = all_reports[all_reports['report_id'].str.startswith('ATSB') & (all_reports['investigation_type'] != 'short')]
atsb_reports.set_index('report_id', inplace=True)
atsb_reports

In [ ]:
sample_atsb = atsb_reports.sample(frac=0.1, random_state=42)


sample_dir = 'sample_atsb'

if os.path.exists(sample_dir):
    shutil.rmtree(sample_dir)

os.mkdir(sample_dir)
for report_id in sample_atsb['report_id']:
    if not os.path.exists(f'../../output/report_pdfs/{report_id}.pdf'):
        continue
    shutil.copy(f'../../output/report_pdfs/{report_id}.pdf', f'{sample_dir}/{report_id}.pdf')

sample_atsb

After looking through the entire sample it seems that only abotu 2 reports ahve recommendations. I dont know if this is just because the didnt have recommendation or they are listed elsewhere.else

I am going to have a look into TAIC and ATSBs recommendations datasets to see how many there are per report.

In [ ]:
df = tsb_recommendations
report_groups = [v.reset_index(drop=True) for k, v in df.groupby("report_id")]

widened_df = pd.DataFrame(
    {
        "report_id": df.groupby("report_id").groups.keys(),
        "recommendations": report_groups,
    }
)

all_recommendations = pd.concat([widened_df, taic_recommendations], axis=0)

all_recommendations

In [ ]:
report_titles = pd.read_pickle('../../output/report_titles.pkl')
report_titles['agency'] = report_titles['report_id'].str[:4]
report_titles.value_counts('agency')

I can see that the TSB only has about 100 reports which have recommendations. However TAIC has about 300 reports in that same time period. This means that TAIC is releasing alot more recomendations. This is equally curious as TAIC has only released about 545 reports whereas TSB has done 1500 in that same time period. Meaning that 1/15 TSB reprots has recommendations but taic is about 1/2. About 8 times more. I know that alot of tSB reports are short rpeorts so that will account for some but not all of this difference.

In [ ]:
sample_atsb['content_section'] = sample_atsb.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id'], x['headers']).extract_contents_section(), axis=1    
)

In [ ]:
importlib.reload(ReportExtracting)
sample_atsb['recommendation_sections'] = sample_atsb.progress_apply(
    lambda x: ReportExtracting.RecommendationsExtractor(x['text'], x['report_id'], x['headers'])._extract_recommendation_section_text(), axis=1
)
sample_atsb.to_pickle("sample_atsb_df")

## Cost ot read all of safety actions sections

In [ ]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o')

sample_atsb['recommendation_section_tokens'] = sample_atsb['recommendation_sections'].progress_apply(
    lambda x: len(encoder.encode(x)) if x else 0
)

sample_atsb.describe()

In [ ]:
(len(atsb_reports) * sample_atsb['recommendation_section_tokens'].mean() / 1_000) * 0.00250

# Reading the safety action section
Now that the safety action sections have been extracted I can read them with the use of an LLM. It will be a simple query just to extract the recommendations.

In [ ]:
importlib.reload(ReportExtracting)
sample_atsb['recommendations'] = sample_atsb.progress_apply(
    lambda x:  ReportExtracting.RecommendationsExtractor(x['text'], x['report_id'], x['headers'])._extract_recommendations_from_text(x['recommendation_sections']) if x['recommendation_sections'] else None, axis=1
)
sample_atsb.to_pickle("sample_atsb_df")

In [ ]:
sample_atsb = pd.read_pickle("sample_atsb_df")
sample_atsb

## Creating the test data

In [ ]:
test_data = pd.read_pickle("recommendation_test_data.pkl")
test_data

In [ ]:
test_data = sample_atsb.set_index('report_id').loc[
    ["ATSB_a_2002_780", "ATSB_m_2005_215", "ATSB_a_2021_005", "ATSB_m_2008_012", "ATSB_r_2015_007", # For reading recommendation_sections
     "ATSB_m_2001_163", "ATSB_a_2002_710", "ATSB_r_2014_024", "ATSB_m_2006_234", "ATSB_r_2004_004", "ATSB_a_2003_980", "ATSB_a_2017_105"] # For extracting pages to read
    ][["text", "headers", "content_section", "recommendation_sections", "recommendations"]]
test_data

In [ ]:
test_data.loc['ATSB_m_2008_012', 'recommendations'] = None
test_data.loc['ATSB_a_2002_780', 'recommendations'] = None
test_data

In [ ]:
test_data.rename(columns={"recommendation_sections": "recommendation_section"}).to_pickle("recommendation_test_data.pkl")

## Complete run through for ATSB

In [ ]:
importlib.reload(ReportExtracting)
atsb_reports['recommendations'] = atsb_reports.progress_apply(
    lambda x:  ReportExtracting.RecommendationsExtractor(x['text'], x.name, x['headers']).extract_recommendations(), axis=1
)
atsb_reports.to_pickle("atsb_recommendations")

In [ ]:
atsb_recommendations = pd.read_pickle("atsb_recommendations")

atsb_recommendations

In [ ]:

atsb_recommendations['recommendations'].dropna().apply(lambda x: len(x) if isinstance(x, list) else 0).describe()

Having a look at this I need to compare it to the TAIC and the TSB reports.

In [ ]:
tsb_recommendations = pd.read_pickle('../../output/tsb_website_recommendations.pkl')
tsb_recommendations = pd.DataFrame({
    'report_id': tsb_recommendations.groupby('report_id').groups.keys(),
    'recommendations': [group.reset_index(drop=True) for _, group in tsb_recommendations.groupby('report_id')]
}
)
display(tsb_recommendations['recommendations'].map(len).describe())
taic_recommendations = pd.read_pickle('../../output/taic_website_recommendations.pkl')
taic_recommendations['year'] = taic_recommendations['report_id'].map(lambda x: int(x.split('_')[2]))
taic_recommendations = taic_recommendations[taic_recommendations['year'] >= 2000]
display(taic_recommendations['recommendations'].map(len).describe())

Something to note here is that TAIC has a much higher amount of recommendations. It averages about the same amount per report that ARE making recommendations.

However it is much more likely for a report to make a recommendation than for TSB or ATSB to do so. This is partly because all of TAICs reports are full reports where alot of ATSB and TSB are actually short reports.

In [ ]:
report_titles

In [ ]:
report_titles = pd.read_pickle('../../output/report_titles.pkl')
agency_report_counts = report_titles.query("investigation_type != 'short'")['report_id'].map(lambda x: x.split('_')[0]).value_counts()
print(f"TAIC has {agency_report_counts['TAIC']} TAIC reports and {len(taic_recommendations)} reports with recommendations which means it has a make recommendations percentage of {len(taic_recommendations)/agency_report_counts['TAIC']*100:.2f}%")
print(f"TSB has {agency_report_counts['TSB']} TSB reports and {len(tsb_recommendations)} reports with recommendations which means it has a make recommendations percentage of {len(tsb_recommendations)/agency_report_counts['TSB']*100:.2f}%")
print(f"ATSB has {agency_report_counts['ATSB']} ATSB reports  and {len(atsb_recommendations['recommendations'].dropna())} reports with recommendations which means it has a make recommendations percentage of {len(atsb_recommendations['recommendations'].dropna())/agency_report_counts['ATSB']*100:.2f}%")

In [ ]:
recs_per_year = taic_recommendations.groupby('year')['num'].sum()

In [ ]:

old_report_titles = pd.read_pickle('../../output/old_report_titles.pkl')
old_report_titles['agency'] = old_report_titles['report_id'].map(lambda x: x.split('_')[0])
taic_reports = old_report_titles[old_report_titles['agency'] == 'TAIC']
taic_reports['year'] = taic_reports['report_id'].map(lambda x: int(x.split('_')[2]))

reports_per_year = taic_reports.groupby('year')['report_id'].count()


In [ ]:
# Get the recommendations per report by combing teh taic_reports and taic_recommendaitons

recs_per_report = {
    year: recs_per_year[year] / reports_per_year[year] for year in range(2000,2024)
}
df = pd.DataFrame(zip(recs_per_report.keys(), recs_per_report.values()), columns=['year', 'recommendations_per_report'])
df



In [ ]:
from plotnine import ggplot, aes, geom_bar, theme_minimal, labs

chart = (ggplot(, aes(x='year', y='recommendation_count')) +
         geom_bar(stat='identity') +
         theme_minimal() +
         labs(title='Recommendations per Report by Year', x='Year', y='Recommendations per Report'))
chart

# Intergrating it with the report extracting processor

The reommendation extraction process needs to be updated to be included into the pipeline.

It needs to still output a recommendation file that is wide with one row per report that has recommendations.recommendation_scraper

TSB still hasnt got oteh code to convert the agnecy_id to report_id. This is because the new report_titles scrape hasnt been done.

In [ ]:
report_titles = pd.read_pickle('../../output/report_titles.pkl')
report_titles

In [ ]:
parsed_reports = pd.read_pickle('../../output/parsed_reports.pkl')

importlib.reload(ReportExtracting)
processor = ReportExtracting.ReportExtractingProcessor('../../output/parsed_reports.pkl', refresh=True)

processor.extract_recommendations("recommendations.pkl", "temp.pkl", "../../output/taic_website_recommendations.pkl")
